In [10]:
# Import libraries
import speech_recognition as sr
import os
import glob
from pydub import AudioSegment
from tqdm import tqdm
from pydub.silence import split_on_silence

path_mp3 = os.path.join('..', 'data', 'raw', 'New Project')
path_wav = os.path.join('..', 'data', 'interim')

# File list
filelist = glob.glob(path_mp3)

# Convert mp3 to wav
for file in tqdm(filelist):
    name, ext = os.path.splitext(file)
    # Filter mp3 extension
    if ext == ".mp3":
        audio_mp3 = AudioSegment.from_file(file)
        audio_mp3.export(os.path.join(path_wav, f"{name}.wav"), format="wav")

# Start speech recognizer
recognizer = sr.Recognizer()

# audio folder
path_wav = './data/interim/'
# output folder
path_text = './data/processed/'

# File list
filelist = glob.glob(path_wav)

# Read wav files
for file in tqdm(filelist):
    name, ext = os.path.splitext(file)
    # Filter mp3 extension
    if ext == ".mp3":    
        audio_mp3 = AudioSegment.from_file(file)
        output_path = os.path.join(path_wav, f"{name}.wav")
        audio_mp3.export(output_path, format="wav")
        
        # split audio when silence is longer than 700 milliseconds
        chunks = split_on_silence(audio_file, min_silence_len=500, silence_thresh=audio_file.dBFS - 14, keep_silence=500)

        chunks_folder = "audio-chunks"

        # create folder to save audio chunks
        if not os.path.isdir(chunks_folder):
            os.mkdir(chunks_folder)

        full_text = ""

        # process every chunk
        for i, audio_chunk in enumerate(chunks, start=1):
            # export chunk and save
            chunks_file = os.path.join(chunks_folder, f"chunk{i}.wav")
            audio_chunk.export(chunks_file, format="wav")
            # Speech recognition
            with sr.AudioFile(chunks_file) as source:
                recognizer.adjust_for_ambient_noise(source)  # Adjusts to eliminate ambient sound
                audio = recognizer.record(source)  # Read audio
                try:
                    text = recognizer.recognize_google(audio, language='es-CL')  # Text recognized
                    # Export to text file
                except sr.UnknownValueError:
                    print("Audio not understandable")
                except sr.RequestError as e:
                    print("No requirement obtained {0}".format(e))
                else:
                    text = f"{text.capitalize()}. "
                    full_text += text
                    with open(path_text+"{0}.txt".format(name), mode='w') as file:
                        file.write(full_text)
                        file.write("\n")  # Corrected line


0it [00:00, ?it/s]
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 5269.23it/s]
